<a href="https://colab.research.google.com/github/nirb28/ee-predict/blob/main/cd_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
def resolve_path_gdrive(relativePath):
    if os.path.exists('/content/drive'):
        return '/content/drive/MyDrive/work/gdrive-workspaces/git/Zahrt/src-model/' + relativePath
    else:
        return relativePath
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np, os, csv
import warnings
from sklearn.metrics.pairwise import euclidean_distances
warnings.filterwarnings('ignore')

In [132]:
## All of these are tunable paramaters
MODEL_REFRESH_STRATEGY = "RETRAIN" # Retrain model after every loop: RETRAIN, PRETRAIN
OPTIMIZE_STRATEGY = "NEWLY_ADDED" # Only optimize the newly added catalyst: NEWLY_ADDED, ALL
MODEL_FIT_USE_COLS = "models/high_corr_cols.txt" # Use all the columns: "ALL", "models/high_corr_cols.txt"
DEFAULT_PCA_DIMENSION = 4
START_SAMPLE_SIZE = 3
DATAFILE_NAME = "merged_large_catalyst.csv" # "reduced_dim_space_ddG.csv", "merged_large_catalyst.csv"
CD_STEP_SIZE = 0.001
CD_STEP_SIZE_MULTIPLIER = 1.5
CD_ITERATIONS_PER_STEP = 2
CD_ITERATIONS = 10
max_cycles = 50
distance_threshold = 2.75
## *************************************************

In [19]:
df = pd.read_csv(resolve_path_gdrive('data/' + DATAFILE_NAME))
## removing ddG = 0 and indexing catalysts
df = df[df['ddG'] != 0]
df.set_index('Catalyst', inplace=True)

if DATAFILE_NAME == "reduced_dim_space_ddG.csv":
  X_COLS = ['x', 'y', 'z']
else:
  if MODEL_FIT_USE_COLS == "ALL":
    X_COLS = df.columns[~df.columns.isin(['Catalyst', 'ddG'])]
  else:
    with open(resolve_path_gdrive(MODEL_FIT_USE_COLS)) as f:
      reader = csv.reader(f)
      X_COLS = list(reader)[0][:-1]
df.head()

,1,2,3,4,5,6,7,8,9,10,...,3966,3967,3968,3969,3970,3971,3972,3973,3974,ddG
Catalyst,,,,,,,,,,,,,,,,,,,,,
1_1_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633,-0.672194,1.226289
1_1_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.602624,-0.471616,-0.798242,-0.277709,-0.280445,-0.279338,-0.246094,-1.153164,-1.146684,0.719002
1_11_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633,-0.672194,0.631877
1_2_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633,-0.672194,0.216792
1_2_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.602624,-0.471616,-0.798242,-0.277709,-0.280445,-0.279338,-0.246094,-1.153164,-1.146684,0.111677


In [20]:
# method to make a regression model from the provided catalysts
from sklearn.cross_decomposition import PLSRegression
def make_pls_model(df_catalysts, PCA_DIMENSION):
    X = df_catalysts[X_COLS]
    y = df_catalysts['ddG']
    model = PLSRegression(n_components=PCA_DIMENSION)
    print("PLS: Fitting model with {} rows and {} columns. PCA dimensions are {}"
        .format(len(X), len(X.columns), PCA_DIMENSION))
    # fitting the model
    return model.fit(X, y)

## Importing model created in "eda.ipynb"
from joblib import dump, load
def load_model(path):
  return load(path)

pls_large_saved_model = load_model(resolve_path_gdrive('models/pls_large.joblib'))
print(pls_large_saved_model)
## Read the dimension directly from the model so that if the model changes
## the dimension will be reflected automatically
PCA_DIMENSION = pls_large_saved_model.n_components

from sklearn.linear_model import LinearRegression
def make_lr_model(df_catalysts):
    X = df_catalysts[X_COLS]
    y = df_catalysts['ddG']
    model = LinearRegression()
    print("LR: Fitting model with {} rows and {} columns."
        .format(len(X), len(X.columns)))
    # fitting the model
    return model.fit(X, y)

def make_model(df_catalysts, PCA_DIMENSION=DEFAULT_PCA_DIMENSION):
    return make_pls_model(df_catalysts, PCA_DIMENSION)
    #return make_lr_model(df_catalysts)

def predict_ee(model, properties):
    return model.predict(properties.reshape(1,-1))

## Add a predicted ddG column to the df
#df['pred_ddG'] = df.apply(lambda x: predict_ee(pls_large_saved_model, np.array(x[X_COLS]))[0][0], axis=1)

PLSRegression(n_components=4)


In [21]:
# start with a few randomly selected catalyst
random_catalyst_df = df.sample(START_SAMPLE_SIZE)
## This will be used to track progress of coordinate descent
print("The max value (target) of the ddG is at {}. Currently we have reached {}.".format(df['ddG'].max(), random_catalyst_df['ddG'].max()))
random_catalyst_df

The max value (target) of the ddG is at 1.640249758. Currently we have reached 0.526005382.


,1,2,3,4,5,6,7,8,9,10,...,3966,3967,3968,3969,3970,3971,3972,3973,3974,ddG
Catalyst,,,,,,,,,,,,,,,,,,,,,
7_11_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.605952,-0.4717,-0.76848,-0.178536,-0.29859,-0.23864,-0.27751,-0.230633,-0.672194,0.526005
21_11_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.605952,-0.4717,-0.76848,-0.178536,-0.29859,-0.23864,-0.27751,-0.230633,-0.672194,0.261200
29_2_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.605952,-0.4717,-0.76848,-0.178536,-0.29859,-0.23864,-0.27751,-0.230633,-0.672194,0.300024


In [22]:
original_ee = df['ddG']
# Function to optimize catalyst properties using coordinate descent
def optimize_catalysts(catalysts, pls_model=None, iterations=CD_ITERATIONS,
                       cd_iterations=CD_ITERATIONS_PER_STEP, step_size=CD_STEP_SIZE):
    print("Optimizing {} catalysts with step size {}.".format(len(catalysts), step_size))
    if pls_model == None:
        pls_model = make_model(random_catalyst_df)
    optimized_catalysts = np.copy(catalysts)
    for _ in range(iterations):
        for i in range(len(optimized_catalysts)):
            ## we have few options for the original_ee. We can use from the dataset or get the prediction
            ## from the model. The prediction makes sense as we want to traverse in the direction where
            ## ee improves according to the model

            ## when using ddG from dataset
            original_ee = df.iloc[df.index.get_loc(catalysts[i:i+1].index[0])]['ddG']
            ## when using the model
            #original_ee = predict_ee(pls_model, optimized_catalysts[i])
            for x in range(len(optimized_catalysts[i])):
                for cd in range(cd_iterations):
                    old_value = optimized_catalysts[i, x]
                    optimized_catalysts[i, x] = old_value + step_size
                    new_ee = predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value - step_size
                        new_ee = predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value
                        break
                    #print('Optimize: Found a direction for higher ddG')
    return optimized_catalysts


In [23]:
if MODEL_REFRESH_STRATEGY == "RETRAIN":
  new_optimized_catalysts = optimize_catalysts(random_catalyst_df[X_COLS])
else:
  new_optimized_catalysts = optimize_catalysts(random_catalyst_df[X_COLS], pls_large_saved_model)

Optimizing 3 catalysts with step size 0.001.
PLS: Fitting model with 3 rows and 62 columns. PCA dimensions are 4


In [24]:
# Perform k-nearest neighbors analysis
def recomputeKNN(df, removePointsDF = None):
  if len(removePointsDF) > 0:
    df.drop(removePointsDF.index, axis=0, inplace=True)
  num_neighbors = 1  # Number of neighbors to consider
  knn = NearestNeighbors(n_neighbors=num_neighbors)
  print('KNN: Fitting model with {} points.'.format(len(df)))
  knn.fit(df)  # Using all the catalysts
  return knn

knn = recomputeKNN(df[X_COLS], random_catalyst_df)

KNN: Fitting model with 316 points.


In [25]:
## Recursive Loop
## The method will get unique neighbors that are not already in the list.
## If the 1st neighbor is already in the list then we try the 2nd, 3rd and so on
## till a unique neighbor is found
def find_unique_neighbors_add(knn, point, addToDf, neighbor_number=1):
    distances, indices = knn.kneighbors([point], neighbor_number)
    neighbor_catalyst = df.iloc[[indices[0][neighbor_number-1]]]
    if neighbor_catalyst.index[0] in addToDf.index:
        #get next neighbor
        neighbor_catalyst = find_unique_neighbors_add(knn, point, addToDf, neighbor_number+1)
    return neighbor_catalyst

def process_neighbors(knn, for_points, addToDf):
    for i in range(len(for_points)):
        neighbor_catalyst = find_unique_neighbors_add(knn, for_points[i], addToDf)
        ## Add found catalysts into our batch (random_catalyst_df)
        if len(neighbor_catalyst) > 0:
            addToDf = pd.concat([addToDf, neighbor_catalyst], ignore_index=False, sort=True)
    return addToDf

random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
random_catalyst_df[X_COLS]

,906,907,908,922,923,937,950,951,952,1113,...,3173,3524,3525,3547,3548,3628,3688,3916,3940,3951
Catalyst,,,,,,,,,,,,,,,,,,,,,
7_11_1,0.066667,0.066667,0.244444,0.377778,0.222222,0.244444,0.000000,0.022222,0.000000,0.266667,...,0.000000,-0.023333,-0.029867,0.00000,0.000000,-0.006667,0.000000,0.056667,0.000000,0.000000
21_11_1,0.200000,0.200000,0.200000,0.600000,0.200000,0.200000,0.200000,0.400000,0.200000,0.200000,...,-0.028800,0.000000,-0.060000,0.00000,0.000000,0.000000,0.000000,0.060000,0.000000,0.000000
29_2_1,0.000000,0.039474,0.105263,0.157895,0.171053,0.105263,0.013158,0.065789,0.131579,0.197368,...,-0.004079,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9_1_2,0.000000,0.000000,0.000000,0.000000,0.039062,0.046875,0.000000,0.000000,0.007812,0.000000,...,0.000000,0.029297,0.058594,-0.01875,-0.022266,0.000000,0.038672,-0.076594,-0.017578,0.007031
21_11_2,0.200000,0.200000,0.200000,0.600000,0.200000,0.200000,0.200000,0.400000,0.200000,0.200000,...,-0.028800,0.000000,-0.060000,0.00000,0.000000,0.000000,0.000000,0.060000,0.000000,0.000000
15_2_1,0.000000,0.040541,0.081081,0.135135,0.256757,0.121622,0.000000,0.081081,0.162162,0.175676,...,-0.004054,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [95]:
# We will now do all this in a single loop

## Maximum of entire dataset
max_ddG = df['ddG'].max()
print("The max value (target) of the ddG is {}. Model training strategy is {}".format(max_ddG, MODEL_REFRESH_STRATEGY))

## Top 5 ddG of entire dataset
top5_ddg = df.sort_values('ddG', ascending=False).head(5)
top5_ddG_values = top5_ddg['ddG']

## Get random rows
random_catalyst_df = df.sample(START_SAMPLE_SIZE)
## Get lowest ddG to start
#random_catalyst_df = df.sort_values('ddG').head(START_SAMPLE_SIZE)
## Static list to start
prev_max_ddg = 0
use_step_size = CD_STEP_SIZE
while(len(random_catalyst_df) < len(df)):
    current_max_ddg = random_catalyst_df['ddG'].max()
    print("The current maximum ddg is at: " + str(current_max_ddg))
    if prev_max_ddg == current_max_ddg:
        print("ddG has not increased, increasing step size")
        use_step_size *= CD_STEP_SIZE_MULTIPLIER
    else:
        use_step_size = CD_STEP_SIZE
    what_to_optimize = random_catalyst_df[X_COLS]
    if OPTIMIZE_STRATEGY == "NEWLY_ADDED":
        ## for newly_added strategy, we only need the last added rows (tail)
        what_to_optimize = what_to_optimize.tail(START_SAMPLE_SIZE)
    new_optimized_catalysts = optimize_catalysts(what_to_optimize,
        pls_large_saved_model if MODEL_REFRESH_STRATEGY == "PRETRAIN" else None, step_size=use_step_size)
    random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
    # recompute knn so that we dont find the same neighbors in the next loop around
    knn = recomputeKNN(df[X_COLS], random_catalyst_df)
    if random_catalyst_df['ddG'].max() >= min(top5_ddG_values):
        print("Found the highest ddG {}. Exiting.".format(str(random_catalyst_df['ddG'].max())))
        break
    prev_max_ddg = current_max_ddg



The max value (target) of the ddG is 1.640249758. Model training strategy is RETRAIN
The current maximum ddg is at: 0.749622551
Optimizing 3 catalysts with step size 0.001.
PLS: Fitting model with 3 rows and 62 columns. PCA dimensions are 4
KNN: Fitting model with 313 points.
The current maximum ddg is at: 1.041413101
Optimizing 3 catalysts with step size 0.001.
PLS: Fitting model with 6 rows and 62 columns. PCA dimensions are 4


KeyboardInterrupt: 

In [ ]:
1# sample prediction with model
properties1 = (np.array([ 0.75813041,  0.67727654,  0.68692902,  0.57403813, -0.39901283,
       -0.83317389, -0.69366434, -0.76423736, -0.18775295, -0.89697628,
       -0.93897206, -0.64303136, -0.50508299, -0.82567658, -0.29773217,
       -0.23605932, -0.26722481,  0.02879822, -0.69189841]))
#4
predict_ee(pls_large_saved_model, properties1)
# random_catalyst_df = random_catalyst_df[~random_catalyst_df.index.duplicated(keep='first')]

In [99]:
# prompt: get last 3 rows of random_catalyst_df[X_COLS]
random_catalyst_df[X_COLS].head(3)


,906,907,908,922,923,937,950,951,952,1113,...,3173,3524,3525,3547,3548,3628,3688,3916,3940,3951
Catalyst,,,,,,,,,,,,,,,,,,,,,
22_9_1,0.000000,0.000000,0.00000,0.000000,0.181818,0.272727,0.0000,0.0000,0.090909,0.363636,...,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4_11_4,0.030303,0.090909,0.30303,0.454545,0.333333,0.333333,0.0000,0.0000,0.000000,0.303030,...,0.0000,0.0000,-0.013636,0.000000,0.000000,-0.009091,0.000000,0.081818,0.000000,0.000000
10_1_2,0.000000,0.062500,0.06250,0.062500,0.125000,0.125000,0.0625,0.0625,0.187500,0.125000,...,-0.0375,0.0375,0.042188,-0.014063,-0.032813,0.000000,0.042188,-0.087750,-0.023438,0.014063


In [ ]:
## Running algorithm certain number of times

current_cycle = 0
catalysts_made_per_cycle = []

prev_max_ddg = 0
## outer while loop
while current_cycle < max_cycles:
  found_max_ddG = False  # Flag to control the inner loop
  ##random_catalyst_df = df.sample(START_SAMPLE_SIZE)
  random_catalyst_df = select_initial_catalysts(df, distance_threshold)

  ## inner while loop
  while(len(random_catalyst_df) < len(df)):
        current_max_ddg = random_catalyst_df['ddG'].max()
        print("The current maximum ddg is at: " + str(current_max_ddg))
        if prev_max_ddg == current_max_ddg:
            print("ddG has not increased, increasing step size")
            use_step_size *= CD_STEP_SIZE_MULTIPLIER
        else:
            use_step_size = CD_STEP_SIZE
        what_to_optimize = random_catalyst_df[X_COLS]
        if OPTIMIZE_STRATEGY == "NEWLY_ADDED":
            ## for newly_added strategy, we only need the last added rows (tail)
            what_to_optimize = what_to_optimize.tail(START_SAMPLE_SIZE)
        new_optimized_catalysts = optimize_catalysts(what_to_optimize,
            pls_large_saved_model if MODEL_REFRESH_STRATEGY == "PRETRAIN" else None, step_size=use_step_size)
        random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
        # recompute knn so that we dont find the same neighbors in the next loop around
        knn = recomputeKNN(df[X_COLS], random_catalyst_df)
        if random_catalyst_df['ddG'].max() >= min(top5_ddG_values):
            print("Found the highest ddG {}. Exiting.".format(str(random_catalyst_df['ddG'].max())))
            print(len(random_catalyst_df))
            catalysts_made_per_cycle.append(len(random_catalyst_df))
            found_max_ddG = True  # Set flag to true to break inner loop
            break
        prev_max_ddg = current_max_ddg


  current_cycle += 1
  if found_max_ddG:
        continue  # Move to the next cycle if max ddG was found

  if current_cycle == max_cycles:
    print("Maximum cycles reached. Exiting.")
    break






The current maximum ddg is at: 1.541190033
Optimizing 3 catalysts with step size 0.001.
PLS: Fitting model with 3 rows and 62 columns. PCA dimensions are 4
KNN: Fitting model with 313 points.
The current maximum ddg is at: 1.541190033
ddG has not increased, increasing step size
Optimizing 3 catalysts with step size 0.0015.
PLS: Fitting model with 6 rows and 62 columns. PCA dimensions are 4
KNN: Fitting model with 310 points.
The current maximum ddg is at: 1.541190033
ddG has not increased, increasing step size
Optimizing 3 catalysts with step size 0.0022500000000000003.
PLS: Fitting model with 9 rows and 62 columns. PCA dimensions are 4
KNN: Fitting model with 307 points.
The current maximum ddg is at: 1.541190033
ddG has not increased, increasing step size
Optimizing 3 catalysts with step size 0.0033750000000000004.
PLS: Fitting model with 12 rows and 62 columns. PCA dimensions are 4
KNN: Fitting model with 304 points.
The current maximum ddg is at: 1.541190033
ddG has not increased, 

In [131]:
## Finding average catalysts made

catalysts_made_per_cycle
sum_catalysts_made = 0

for i in catalysts_made_per_cycle:
  sum_catalysts_made += i

average_catalysts_made = sum_catalysts_made / len(catalysts_made_per_cycle)

print(catalysts_made_per_cycle)
print("The average number of catalysts made are " + str(average_catalysts_made))
print("The standard deviation is " + str(np.std(catalysts_made_per_cycle)))
Standard_Error_95 = np.std(catalysts_made_per_cycle) / np.sqrt(len(catalysts_made_per_cycle))
print("The standard error*2 is " + str(Standard_Error_95*2))
print("The number of trials run are " + str(len(catalysts_made_per_cycle)))


[6, 42, 36, 78, 30, 66, 27, 45, 42, 9, 42, 60, 75, 24, 87, 60, 18, 39, 72, 33, 75, 12, 33, 15, 12, 36, 30, 45, 27, 30, 72, 24, 6, 120, 33, 48, 24, 81, 30, 48, 33, 24, 9, 96, 15, 30, 39, 30, 96, 30]
The average number of catalysts made are 41.88
The standard deviation is 26.00818332756058
The standard error*2 is 7.356225118904397
The number of trials run are 50


In [103]:
 ## Check distance mechanism
check_distance(random_catalyst_df.head(3), distance_threshold)

True

In [111]:

def select_initial_catalysts(catalysts_df, distance_threshold):
    # Extract features as a 2D numpy array
    features = catalysts_df[X_COLS].values
    initial_catalysts = []
    candidates = catalysts_df.copy()

    while len(initial_catalysts) < 3:
        if len(initial_catalysts) == 0:
            # Select the first catalyst randomly
            idx = np.random.choice(len(catalysts_df))
            initial_catalysts.append(catalysts_df.iloc[idx])
        else:
            # Create a mask to keep track of valid candidates
            mask = np.ones(len(candidates), dtype=bool)
            for selected in initial_catalysts:
                selected_features = selected[X_COLS].values.reshape(1, -1)
                for i in range(len(candidates)):
                    candidate_features = features[i].reshape(1, -1)
                    candidate_distance = euclidean_distances(candidate_features, selected_features).flatten()[0]
                    if candidate_distance <= distance_threshold:
                        mask[i] = False  # Mark the candidate as invalid if too close

            # Filter candidates using the mask
            valid_candidates = candidates[mask]

            if valid_candidates.empty:
                mask = True
                del initial_catalysts[0:1]
            else:
                # Select a random candidate from the filtered list
                idx = np.random.choice(len(valid_candidates))
                initial_catalysts.append(valid_candidates.iloc[idx])

    return pd.DataFrame(initial_catalysts)

initial_catalysts_df = select_initial_catalysts(df, distance_threshold)
initial_catalysts_df

,1,2,3,4,5,6,7,8,9,10,...,3966,3967,3968,3969,3970,3971,3972,3973,3974,ddG
29_3_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.643031,-0.505083,-0.825677,-0.297732,-0.236059,-0.267225,0.028798,-0.691898,-0.672194,0.049780
20_1_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.602624,-0.471616,-0.798242,-0.277709,-0.280445,-0.279338,-0.246094,-1.153164,-1.146684,1.092551
9_9_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633,-0.672194,0.476662


In [71]:
def check_distance(initial_catalysts_df, distance_threshold):
  features = initial_catalysts_df[X_COLS].values
  for i in range(len(initial_catalysts_df)):
    for j in range(i+1, len(initial_catalysts_df)):
      distance = euclidean_distances(features[i].reshape(1,-1), features[j].reshape(1,-1)).flatten()[0]
      if distance < distance_threshold:
        return False
  return True
result = check_distance(initial_catalysts_df, distance_threshold)

if result:
  print("All catalysts are at least {} apart from each other.".format(distance_threshold))
else:
  print("Some catalysts are closer than {} apart.".format(distance_threshold))


All catalysts are at least 2.5 apart from each other.


In [ ]:
## Computing max and average distances between pairs of catalysts in chemical space

def max_distance(df, cols):

  features = df[cols].values
  distances = euclidean_distances(features, features)
  return np.max(distances)

max_distance = max_distance(df, X_COLS)
print(f"Maximum distance between any two catalysts: {max_distance}")

def avg_distance(df, cols):

  features = df[cols].values
  distances = euclidean_distances(features, features)
  return np.mean(distances)

avg_distance = avg_distance(df, X_COLS)
print(f"Average distance between all pairs of catalysts: {avg_distance}")


Maximum distance between any two catalysts: 4.561734984971301
Average distance between all pairs of catalysts: 2.0438268534236603


In [125]:
##

def global_search(dataframe, current_dataset, num_new_catalysts = 1):
    new_catalysts = []
    features = dataframe[X_COLS].values
    while len(new_catalysts) < num_new_catalysts:
        candidates = dataframe.copy()
        for i in range(len(candidates)):
            selected_features = features[i].reshape(1, -1)
            candidate_distance = euclidean_distances(selected_features, current_dataset[X_COLS].values).flatten()[0]


global_search(df[X_COLS], initial_catalysts_df)


KeyboardInterrupt: 